# Übung 6: Transfer learning

Neuronal Netze benötigen oft eine große Menge an Trainingsdaten, damit es nicht zu overfitting kommt. Transfer learning erlaubt es, mit relativ geringen Datenmenge dennoch erfolgreiche große Netze zu trainieren. Dabei verwendet man ein bereits auf einen anderen Datensatz (z.b. ImageNet) vortrainiertes Netzwerk, und ersetzt nur das letzte Layer durch ein neues. In dieser Übung geht es darum, ein Netzwerk für die Erkennung von Geparden und Leoparden in der freien Wildbahn zu trainineren. 

## Daten laden

Lade die Daten hier herunter: http://tonic.imp.fu-berlin.de/cv_data/data.tar.gz

Die Daten wurde bereits in ein Trainings- und Validierungsset geteilt. Die Ordnerstruktur ist wie bei vielen Bildklassifierungsdatensetzen so aufgebaut. Es gibt zwei Unterordner für die Trainings- und Validierunsdaten. In diesen Ordnern liegen dann jeweils alle Bilder von einer Klasse in einem Unterordner mit dem Namen der Klasse.

Ein Beispiel: Die Trainingsbilder für die Klasse "cheetah" liegen in dem Unterordner train/cheetah

Diese Orderstruktur wird auch von dem in keras enhaltenen ImageDataGenerator unterstützt.

In [1]:
import os
import tensorflow as tf
import numpy as np
import keras
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

from keras.preprocessing.image import ImageDataGenerator
#
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras import optimizers
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import load_model
from keras.models import Model

from PIL import Image

Using TensorFlow backend.


In [2]:
batch_size = 32
image_input_size = (224, 224)
data_path = './data/'

prob_drop_conv = 0.5              
pool_size = (2, 2) 
nb_epoch = 150
opt = optimizers.RMSprop(lr = 0.000001, rho = 0.9)

In [3]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')

izw_classes = ('unknown', 'cheetah', 'leopard')

generator = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
val_generator = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False)

train_gen = generator.flow_from_directory(
    train_data_path, 
    target_size = image_input_size,
    classes = izw_classes,
    batch_size = batch_size,
    shuffle = False)

val_gen = val_generator.flow_from_directory(
    val_data_path, 
    target_size = image_input_size,
    classes = izw_classes,
    batch_size = batch_size,
    shuffle = False)


Found 17857 images belonging to 3 classes.
Found 1915 images belonging to 3 classes.


## Training ohne transfer learning

Trainiere zuerst ein kleines Classifer-Netzwerk ohne transfer learning. Falls du keine Grafikkarte hast, solltest du nicht die volle Auflösung (siehe Variable image_input_size) verwenden, da das Training sonst zu lange dauert. Eine Bildgröße von 32x32 Pixeln wäre zum Beispiel möglich.

In [4]:
# Convolutional model
model = Sequential()
steps_per_epoch=len(pool_size)/batch_size

# conv1 layer
model.add(Convolution2D(8, 3, 3, border_mode = 'same', activation = 'relu', input_shape = [224,224,3]))
model.add(MaxPooling2D(pool_size = pool_size, strides=(2,2), border_mode = 'same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Convolution2D(16, 3, 3, border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size, strides = (2,2), border_mode = 'same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Convolution2D(4, 3, 3, border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size, strides = (2,2), border_mode = 'same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
model.add(Dense(625, activation = 'relu'))
model.add(Dropout(prob_drop_conv))

# fc2 layer
model.add(Dense(3, activation = 'softmax'))

model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()
print(model.summary())

# Train
history = model.fit_generator(train_gen,steps_per_epoch, nb_epoch = nb_epoch, shuffle = False, verbose = 1)



W1210 12:52:58.550036 4529900992 deprecation_wrapper.py:119] From /Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", input_shape=[224, 224,..., padding="same")`
  
W1210 12:52:58.553548 4529900992 deprecation_wrapper.py:119] From /Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1210 12:52:58.556144 4529900992 deprecation_wrapper.py:119] From /Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.unifo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 8)       224       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 16)      1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 4)         580       
__________

Epoch 9/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 373ms/step - loss: 4.9300 - acc: 0.6875
Epoch 10/150
1/0 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================

Epoch 25/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 352ms/step - loss: 5.6245 - acc: 0.5938
Epoch 26/150
1/0 [=====================================================================================================================================================================================================================================================================================================================================================================================================================================================

Epoch 41/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 318ms/step - loss: 8.5627 - acc: 0.4688
Epoch 42/150
1/0 [=====================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

Epoch 73/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 408ms/step - loss: 5.5406 - acc: 0.6562
Epoch 74/150
1/0 [=====================================================================================================================================================================================================================================================================================================================================================================================================================================================

Epoch 89/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 357ms/step - loss: 5.5407 - acc: 0.6562
Epoch 90/150
1/0 [=====================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 326ms/step - loss: 7.3380 - acc: 0.5312
Epoch 105/150
1/0 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Epoch 120/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 422ms/step - loss: 6.0443 - acc: 0.6250
Epoch 121/150
1/0 [===================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 377ms/step - loss: 4.5332 - acc: 0.7188
Epoch 136/150
1/0 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [5]:
# Evaluate
evaluation = model.evaluate_generator(val_gen, 1)

print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

Summary: Loss over the test dataset: 0.00, Accuracy: 1.00


    Erstelle eine Confusion matrix basierend auf den Ausgaben des Netzes für die Validierungsdaten und berechne den ROC AUC für die Klasse cheetah. Du kannst hierfür optional die scikit-learn Bibliothek verwenden.

In [6]:
num_of_test_samples = 1915

Y_pred = model.predict_generator(val_gen, num_of_test_samples // batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
conf_matrix = confusion_matrix(val_gen.classes, y_pred)
print(conf_matrix)

print('')

print('Classification Report')
print(classification_report(val_gen.classes, y_pred, target_names=['unknown', 'cheetah', 'leopard']))

print('')

print('ROC AUC Score')
0.75

Confusion Matrix
[[ 359    5    0]
 [1330  110    0]
 [ 109    2    0]]

Classification Report
              precision    recall  f1-score   support

     unknown       0.20      0.99      0.33       364
     cheetah       0.94      0.08      0.14      1440
     leopard       0.00      0.00      0.00       111

    accuracy                           0.24      1915
   macro avg       0.38      0.35      0.16      1915
weighted avg       0.74      0.24      0.17      1915


ROC AUC Score


/Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.75

## Pretrained network

Lade nun ein auf Imagenet vortrainiertes Netzwerk und klassifiziere damit die Validierungsdaten. Eine Anleitung für keras findest du hier: https://keras.io/applications

Du kannst selber entscheiden, welche Netzwerkarchitektur du verwendest.

In [ ]:
model2 = ResNet50(weights = 'imagenet')
evaluation2 = model2.predict_generator(val_gen, steps = (len(val_gen)), verbose = 1)
print(evaluation2)
#x = np.expand_dims(val_gen, axis=0)
#preds = model2.predict(x)

#print('Predicted:', decode_predictions(preds, top=3))

W1210 12:54:09.193561 4529900992 deprecation_wrapper.py:119] From /Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



43/60 [====================>.........] - ETA: 43s

In [ ]:
#TODO DELETE THIS
model2.summary()

In [ ]:
preds = decode_predictions(evaluation2, top=3)
print('Predicted:', preds)

In [ ]:
model2.summary()

Da der ImageNet-Datensatz auch die Klassen cheetah und leopard enthält, können wir sogar ohne transfer learning das vortrainierte Netzwerk evaluieren. Interpretiere alle Klassen außer cheetah und leopard als unknown und berechne wie im vorherigen Schritt die Confusion matrix und den ROC AUC score für die Klasse cheetah.

In [ ]:
for img in preds:
    if img[0][1] not in izw_classes:
        img[0] = (img[0][0],'unknown',img[0][2])

for img in preds:
    print(img)

## Transfer learning

Das vortrainierte Netzwerk kann nun mit unseren Daten weitertrainiert werden. Ersetze dafür das letzte Layer in dem Netzwerk mit einem Dense Layer mit 3 Ausgaben für unsere Klassen cheetah, leopard und unknown. Du kannst selbst entscheiden, ob du nun das komplette Netzwerk mit trainierst oder nur das neu eingefügte, letzte Layer.

Auch hierfür kannst du dich wieder an der keras Anleitung orientieren: https://keras.io/applications

In [ ]:
model2.summary()

In [ ]:
for layer in model2.layers:
    layer.trainable = False
model2.layers.pop()
#print(model2.layers.pop())
model2.layers.pop()
model2.summary()
print(model2.layers)
model2.layers[-1].outbound_nodes = []
model2.outputs = [model2.layers[-1].output]
output1 = model2.get_layer('avg_pool').output
output2 = Flatten()(output2)
output3 = Dense(output_dim=3)(output3)
model3 = Model(model2.input,output3)
model3.summary()

Evaluiere das so trainierte Netzwerk wie in den letzten beiden Aufgaben.

In [ ]:
model3.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history2 = model3.fit_generator(train_gen, steps_per_epoch = 100, nb_epoch = nb_epoch, shuffle = False, verbose = 1)

# Auswertung

Beschreibe kurz qualitativ die Resultate. Wie unterscheiden sich die trainierten Netzwerke, zum Beispiel im Bezug auf die Genauigkeit oder die Laufzeit? Welche Entscheidungen musstest du bei der Erfüllung der Aufgaben treffen und warum hast du dich für den von dir gewählten Weg entschieden?